# Face Tracker using Handle

This ipython file has two Thread classes. One is controlling jetbot's head and motors with handle(joystick) and the other is tracking faces using face detector.

## Remote control thread

In [1]:
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat
from servoserial import ServoSerial

import os
import cv2
import PID # PID Algorithm
import threading
import time
# Thread function operation library
import inspect
import ctypes

import traitlets
import ipywidgets.widgets as widgets

from IPython.display import display
from uuid import uuid1
from mtcnn.mtcnn import MTCNN

/home/jetbot/Projects/venv/jetbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jetbot/Projects/venv/jetbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jetbot/Projects/venv/jetbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jetbot/Projects/venv/jetbot/lib/python3.6/s

To safely stop threads, we need codes below

In [2]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)

def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

Load usb controller. You need to connect controller to host PC

Before you connect, you need to check controller index through this [link](https://html5gamepad.com/)

In [3]:
controller = widgets.Controller(index=1)  #Replace with the index number of the controller you just tested
display(controller)

Controller(index=1)

If your controller connected properly, you'll see a control widget like above. Check if it changes you push buttons

And then initialize pid camera instance

In [11]:
# camera instance
camera = Camera.instance(width=300, height=300)
image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Open Servo Serial

In order to use servo motor which correnponds to jetbot head, you need to connect to servo serial. If it opens properly, "serial Open!" will be printed

In [5]:
servo_serial = ServoSerial()

serial Open!


### Define Handle Thread

Codes below are JetBot Controlling thread class which can move left, right, up, down freely.

In [6]:
class Handle_Control(threading.Thread):
    def __init__(self, message, controller, servo, index=0, camera_link=None):
        threading.Thread.__init__(self)
        
        self.message = message
        
#         self.controller = widgets.Controller(index = index)
#         print(self.controller.axes)
#         display(self.controller)

        self.controller = controller
        
        print('Waiting for cotroller...')
        while len(self.controller.axes) == 0:
            time.sleep(1)
        print('Controller connected!')
        
        self.camera_link = camera_link
        
        if self.camera_link != None:
            heartbeat = Heartbeat(period=0.5)
            # Append the callback function to the heartbeat state
            heartbeat.observe(self.handle_heartbeat_status, names='status')
        
        '''Servo Part'''
        if servo == None:
            raise ValueError('servo serial must not be None')
        self.servo_device = servo
        self.updownpulse = 2700
        self.leftrightpulse = 2300
        self.camservoInitFunction()
        
        ''' Robot Part '''
        self.robot = Robot()
        
        
    def handle_heartbeat_status(self, change):
        if change['new'] == Heartbeat.Status.dead:
            self.camera_link.unlink()
            self.robot.stop()
        
    def camUpFunction(self):
        self.updownpulse += 15
        if self.updownpulse > 4095:
            self.updownpulse = 4095
        self.servo_device.Servo_serial_control(2, self.updownpulse)
    
    def camDownFunction(self):
        self.updownpulse -= 15
        if self.updownpulse < 1300:
            self.updownpulse = 1300
        self.servo_device.Servo_serial_control(2, self.updownpulse)
        
    def camLeftFunction(self):
        self.leftrightpulse += 15
        if self.leftrightpulse > 3600:
            self.leftrightpulse = 3600
        self.servo_device.Servo_serial_control(1, self.leftrightpulse)
        
    def camRightFunction(self):
        self.leftrightpulse -= 15
        if self.leftrightpulse < 600:
            self.leftrightpulse = 600
        self.servo_device.Servo_serial_control(1, self.leftrightpulse)
        
    def camservoInitFunction(self):
        self.leftrightpulse = 2300
        self.updownpulse = 2700
        self.servo_device.Servo_serial_control(1, self.leftrightpulse)
        time.sleep(0.1)
        self.servo_device.Servo_serial_control(2, self.updownpulse)
        
    def run(self):
        count1 = count2 = count3 = count4 = count5 = 0
        while 1:
            #Robot car Left and right DC motor
#             if self.controller.axes[1].value <= 0.1:
#                 if (self.controller.axes[0].value <= 0.1 and self.controller.axes[0].value >= -0.1
#                     and self.controller.axes[1].value <= 0.1 and self.controller.axes[1].value >= -0.1):
#                     self.robot.stop()
#                 else:
#                     self.robot.set_motors(-self.controller.axes[1].value + self.controller.axes[0].value, -self.controller.axes[1] - self.controller.axes[0].value)

#                 time.sleep(0.01)
#             else:
#                 self.robot.set_motors(-self.controller.axes[1].value - self.controller.axes[0].values, -self.controller.axes[1].value + self.controller.axes[0].value)
#                 time.sleep(0.01)

            if controller.axes[1].value <= 0:
                self.robot.set_motors(-self.controller.axes[1].value + self.controller.axes[0].value, -self.controller.axes[1].value - self.controller.axes[0].value)
                time.sleep(0.01)
            else:
                self.robot.set_motors(-self.controller.axes[1].value - self.controller.axes[0].value, -self.controller.axes[1].value + self.controller.axes[0].value)
                time.sleep(0.01)
                
            # Servo control cameara up and down
            if self.controller.axes[5].value >= 0.5:
                count1 += 1
                if count1 >= 3:
                    self.camDownFunction()
                    count1 = 0
            elif self.controller.axes[5].value <= -0.5:
                count1 += 1
                if count1 >= 3:
                    self.camUpFunction()
                    count1 = 0
            else:
                count1 = 0
                
            # Servo control camera left and right
            if self.controller.axes[2].value <= -0.5:
                count2 += 1
                if count2 >= 3:
                    self.camRightFunction()
                    count2 = 0
            elif self.controller.axes[2].value >= 0.5:
                count2 += 1
                if count2 >= 3:
                    self.camLeftFunction()
                    count2 = 0
            else:
                count2 = 0
                
            # Servo control camera up down, left and right is reset
            if self.controller.buttons[8].value == True:
                count3 += 1
                if count3 >= 3:
                    self.camservoInitFunction()
                    count3 = 0
            else:
                count3 = 0
                
            # Servo control servo rise and decline
            if self.controller.buttons[6].value == True:
                count4 += 1
                if count4 >= 3:
                    self.robot.set_vertical_motors(1)
                    count4 = 0
            elif self.controller.buttons[4].value == True:
                count4 += 1
                if count4 >= 3:
                    self.robot.set_vertical_motors(-1)
                    count4 =0
            else:
                self.robot.set_vertical_motors(0)
                count4 = 0

Let's run jetbot!

In [7]:
handle_thread = Handle_Control(message='Handle', controller=controller, servo=servo_serial, index=0, camera_link=camera_link)
handle_thread.setDaemon(True)
handle_thread.start()

Waiting for cotroller...
Controller connected!
188
b'\xff\xff\x01\x07\x03*\x08\xfc\x00\n\xbc'
41
b'\xff\xff\x02\x07\x03*\n\x8c\x00\n)'


If you want to stop thread, run the code below

In [12]:
stop_thread(handle_thread)

## Face Tracker Thread

### Define detector

In addition to control jetbot with joystick, Jetbot's head can automatically follow human faces through face detector model. 

I used MTCNN(Multi-task Convolutional Neural Network) for face detector. If you need more information of MTCNN, read this [paper](https://arxiv.org/abs/1604.02878)

In [8]:
detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


It takes time to load MTCNN model...

In [13]:
class Face_Tracker(threading.Thread):
    def __init__(self, message, camera, servo, detector, image=None):
        threading.Thread.__init__(self)
        
        self.message = message
        
        ''' camera part '''
        if camera == None:
            raise ValueError("camera instance can\'t be None")
        self.camera = camera
        
        ''' servo part '''
        if servo == None:
            raise ValueError('servo serial must not be None')
        self.servo_device = servo
        self.target_valuex = 2300
        self.target_valuey = 2700
        
        ''' face detector '''
        self.detector = detector
        
        self.image_widget = image
        
        ''' PID control instance '''
        self.xservo_pid = PID.PositionalPID(1.9, 0.4, 0.35)
        self.yservo_pid = PID.PositionalPID(1.5, 0.3, 0.3)
        
        
        self.lock = threading.Lock()
        
    def run(self):
        while 1:
            frame = self.camera.value
            img = frame.copy()
            faces = self.detector.detect_faces(img)
            # if there's faces in the frame
            if len(faces) > 0:
                (face_x, face_y, face_w, face_h) = faces[0]['box'] # get coordinates
                # draw rectangle
                cv2.rectangle(frame, (face_x, face_y), (face_x+face_w, face_y+face_h), color=(0,255,0), thickness=2)
                
                # follow face based on PID algorithm
                self.xservo_pid.SystemOutput = face_x + face_w/2
                self.xservo_pid.SetStepSignal(150)
                self.xservo_pid.SetInertiaTime(0.01, 0.006)
                self.target_valuex = int(2300 + self.xservo_pid.SystemOutput*2)
                
                self.yservo_pid.SystemOutput = face_y + face_h/2
                self.yservo_pid.SetStepSignal(150)
                self.yservo_pid.SetInertiaTime(0.01, 0.006)
                self.target_valuey = int(2700 + self.yservo_pid.SystemOutput*2)
                
                self.servo_device.Servo_serial_double_control(1, self.target_valuex, 2, self.target_valuey)
                
            self.lock.acquire()
            if self.image_widget:
                self.image_widget.value = bgr8_to_jpeg(frame)
            self.lock.release()

In [14]:
thread_face = Face_Tracker('face', camera, servo_serial, detector, None)
thread_face.setDaemon(True)
thread_face.start()

92
b'\xff\xff\xfe\x0e\x83*\x04\x01\t!\x00\n\x02\n\x9b\x00\n\\'
63
b'\xff\xff\xfe\x0e\x83*\x04\x01\t:\x00\n\x02\n\x9f\x00\n?'
72
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x12\x00\n\x02\n\xbe\x00\nH'
78
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xef\x00\n\x02\n\xdc\x00\nN'
129
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x81\x00\n\x02\x0b\x16\x00\n\x81'
176
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x1c\x00\n\x02\x0bL\x00\n\xb0'
210
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xef\x00\n\x02\x0bX\x00\n\xd2'
240
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xdf\x00\n\x02\x0bJ\x00\n\xf0'
245
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xe1\x00\n\x02\x0bC\x00\n\xf5'
251
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xdf\x00\n\x02\x0b?\x00\n\xfb'
254
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xdb\x00\n\x02\x0b@\x00\n\xfe'
254
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xd3\x00\n\x02\x0bH\x00\n\xfe'
2
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xca\x00\n\x02\x0bM\x00\n\x02'
10
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xc0\x00\n\x02\x0bO\x00\n\n'
22
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xb1\x0

In [15]:
stop_thread(thread_face)

108
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07q\x00\n\x02\x0b<\x00\nl'


You need to return camera instance to reuse it

In [15]:
del camera